# <a>Classificação de Imagens - PyTorch e Transfer Learning</a>

Projeto realizado durante a Jornada Cientista de Dados da equipe <i>Let's Data</i>

# <a>Objeto de Estudo</a>

Iremos modelar um Produto Mínimo Viável que possa ser testado nas filiais de uma empresa do ramo de supermercados. O problema a ser solucionado é classificar produtos na hora de colocar no mostruário. Vamos começar com uma classificação simples de batatas, cenouras, tomates e limões.

Para isso iremos usar modelos pré-treinados para nos ajudar com a classificação de imagens através dos padrões.